Cross Validation 4

size =  121655

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 5000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_train_4.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_val_4.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(97324, 23)
(24331, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Science Kids Learning - Be Super Scientist!,com.sciencegames.ScienceKidsLearning,Educational,4.2,78871,False,Science is all about inspiring curiosity and d...,https://nutboltgames.com/privacy-policy,0,0,...,0,0,1,0,0,0,1846,science is all about inspiring curiosity and d...,1807,343
1,Phuti kismat se Bhagyashali,com.photi.bhgysl,Lifestyle,4.4,13681,False,This year has in its fold mixed fate to offer ...,https://labtest12.wixsite.com/privacypolicy,0,0,...,0,0,0,0,0,0,1513,this year has in its fold mixed fate to offer ...,1443,350


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        16397
Location      16234
Microphone     8879
Contacts      10505
Storage       48372
Phone         15199
SMS             326
Call_Log        207
Calendar       1798
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         4056
Location       4108
Microphone     2162
Contacts       2615
Storage       11886
Phone          3678
SMS              87
Call_Log         59
Calendar        443
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

121655


In [23]:
corpus[0]

['science',
 'inspiring',
 'curiosity',
 'discovering',
 'world',
 'learn',
 'science',
 'fun',
 'educational',
 'games',
 'science',
 'games',
 'sure',
 'fulfill',
 'sense',
 'curiosity',
 'introduce',
 'basic',
 'science',
 'concepts',
 'approachable',
 'manner',
 'reinforcing',
 'comprehension',
 'engaging',
 'stimulating',
 'activities',
 'using',
 'interactive',
 'science',
 'games',
 'creates',
 'unique',
 'learning',
 'process',
 'combined',
 'conventional',
 'topics',
 'food',
 'chain',
 'learn',
 'food',
 'chain',
 'food',
 'web',
 'energy',
 'cycles',
 'transfers',
 'living',
 'organisms',
 'lesson',
 'learn',
 'simple',
 'food',
 'chains',
 'energy',
 'passed',
 'producers',
 'consumers',
 'decomposers',
 'eats',
 'what',
 'energy',
 'life',
 'needs',
 'survive',
 'come',
 'from',
 'learn',
 'producers',
 'primary',
 'consumers',
 'secondary',
 'tertiary',
 'consumers',
 'apex',
 'predators',
 'decomposers',
 'roles',
 'play',
 'food',
 'chains',
 'microorganisms',
 'ever',


##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        science inspiring curiosity discovering world ...
1        year fold mixed fate offer fellow human beings...
2        realpi provides best interesting pi calculatio...
3        regularly helps keep track repeating tasks fix...
4        harri live highly intuitive useful team schedu...
                               ...                        
97319    stylist wallpapers keyboard beautiful keyboard...
97320    consists precise easy understand meanings phys...
97321    may light guide dark places pendulum artifact ...
97322    leo browser made minimalism comfortable web br...
97323    boltnet vpn protects internet session helps ac...
Name: Clean_Description, Length: 97324, dtype: object

In [29]:
train_sequences

[[582,
  3039,
  6683,
  6448,
  95,
  31,
  582,
  97,
  458,
  73,
  582,
  73,
  429,
  4752,
  2252,
  6683,
  3309,
  402,
  582,
  1467,
  25963,
  1945,
  20140,
  3805,
  2396,
  8611,
  602,
  39,
  620,
  582,
  73,
  2828,
  191,
  74,
  542,
  2986,
  6096,
  382,
  432,
  3599,
  31,
  432,
  3599,
  432,
  338,
  1015,
  4532,
  1788,
  1632,
  7931,
  1723,
  31,
  47,
  432,
  7057,
  1015,
  4775,
  8127,
  4441,
  57858,
  14145,
  5938,
  1015,
  83,
  418,
  6206,
  513,
  1368,
  31,
  8127,
  2369,
  4441,
  3034,
  24026,
  4441,
  1772,
  12938,
  57858,
  6300,
  67,
  432,
  7057,
  20141,
  496,
  6340,
  3233,
  140,
  12456,
  31832,
  31832,
  31832,
  18090,
  323,
  1903,
  2108,
  155,
  31,
  384,
  1632,
  3955,
  7931,
  18,
  11149,
  5093,
  13815,
  25964,
  34066,
  1172,
  355,
  297,
  2239,
  31,
  1632,
  522,
  1431,
  414,
  1006,
  3,
  404,
  432,
  683,
  180,
  1172,
  41,
  414,
  1006,
  392,
  9096,
  546,
  146,
  95,
  582,
  31,
 

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[  582,  3039,  6683, ...,     0,     0,     0],
       [  267,  7112,  3330, ...,     0,     0,     0],
       [40007,   147,    16, ...,     0,     0,     0],
       ...,
       [  172,   294,   475, ...,     0,     0,     0],
       [ 4908,   579,   260, ...,     0,     0,     0],
       [  213,  3189,    82, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[   40,  8260,   872, ...,     0,     0,     0],
       [   81,   311,   327, ...,     0,     0,     0],
       [  219,  1419,  1537, ...,     0,     0,     0],
       ...,
       [  847,   372,  1635, ...,     0,     0,     0],
       [  908,    17, 14523, ...,     0,     0,     0],
       [   64, 41986,   930, ...,     0,     0,     0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

science inspiring curiosity discovering world learn science fun educational games science games sure fulfill sense curiosity introduce basic science concepts approachable manner reinforcing comprehension engaging stimulating activities using interactive science games creates unique learning process combined conventional topics food chain learn food chain food web energy cycles transfers living organisms lesson learn simple food chains energy passed producers consumers decomposers eats what energy life needs survive come from learn producers primary consumers secondary tertiary consumers apex predators decomposers roles play food chains microorganisms ever wondered happens look microscope trillions trillions trillions microbes around earth maybe more learn types living micro organisms like bacteria viruses fungi algae protozoa human body health growth learn living things grow stay healthy use water food exercise provide human need stay healthy super scientist enter amazing world science

In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 194318


In [36]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'wallpaper': 4,
 'keyboard': 5,
 'theme': 6,
 'phone': 7,
 'application': 8,
 'features': 9,
 'new': 10,
 'wallpapers': 11,
 'get': 12,
 'time': 13,
 'also': 14,
 'screen': 15,
 'best': 16,
 'one': 17,
 'like': 18,
 'download': 19,
 'make': 20,
 'easy': 21,
 'android': 22,
 'launcher': 23,
 'us': 24,
 'english': 25,
 'live': 26,
 'mobile': 27,
 'share': 28,
 'find': 29,
 'help': 30,
 'learn': 31,
 'hd': 32,
 'love': 33,
 'set': 34,
 'video': 35,
 'device': 36,
 'apps': 37,
 'please': 38,
 'using': 39,
 'want': 40,
 'need': 41,
 'images': 42,
 'information': 43,
 'support': 44,
 'themes': 45,
 'home': 46,
 'simple': 47,
 'many': 48,
 'friends': 49,
 'available': 50,
 'enjoy': 51,
 'different': 52,
 'text': 53,
 'data': 54,
 'access': 55,
 'language': 56,
 'day': 57,
 'beautiful': 58,
 'game': 59,
 'easily': 60,
 'online': 61,
 'add': 62,
 'user': 63,
 'choose': 64,
 'way': 65,
 'you': 66,
 'play': 67,
 'create': 68,
 'words': 69,
 'every': 70,
 'status

In [37]:
word_index["reason"]

2423

In [38]:
print(validation_sequences[0])

[40, 8260, 872, 726, 13617, 926, 49, 7, 1428, 943, 2938, 13617, 1046, 156, 2963, 7, 15, 7823, 13617, 486, 570, 13, 64, 1304, 943, 1428, 8260, 483, 5447, 62421, 308, 9, 13617, 7, 1428, 943, 2938, 43756, 64, 1304, 15, 652, 5367, 1692, 284, 64, 3249, 943, 10970, 4296, 3, 7, 674, 1032]


In [39]:
word_index["listen"]

401

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.62230998, -0.40289   ,  0.11949   , ...,  0.93154001,
         0.22446001,  0.15016   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2423

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(97324, 600)
(97324, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(24331, 600)
(24331, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

97324
9
{0: 0.6594973335230699, 1: 0.6661191189957976, 2: 1.2179049192226352, 3: 1.029393410545243, 4: 0.2235544897415401, 5: 0.7114795564035645, 6: 33.17109747784595, 7: 52.24047235641439, 8: 6.014336917562724}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    58295700    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    58295700    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    58295700    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
3042/3042 [==============================] - 561s 181ms/step - loss: 0.1601 - accuracy: 0.2736 - val_loss: 0.2305 - val_accuracy: 0.2852
Epoch 2/10
3042/3042 [==============================] - 549s 180ms/step - loss: 0.1333 - accuracy: 0.2970 - val_loss: 0.2299 - val_accuracy: 0.2773
Epoch 3/10
3042/3042 [==============================] - 548s 180ms/step - loss: 0.1147 - accuracy: 0.3011 - val_loss: 0.2222 - val_accuracy: 0.2778
Epoch 4/10
3042/3042 [==============================] - 548s 180ms/step - loss: 0.0948 - accuracy: 0.3033 - val_loss: 0.2192 - val_accuracy: 0.2808
Epoch 5/10
3042/3042 [==============================] - 554s 182ms/step - loss: 0.0789 - accuracy: 0.3053 - val_loss: 0.2313 - val_accuracy: 0.2708
Epoch 6/10
3042/3042 [==============================] - 553s 182ms/step - loss: 0.0684 - accuracy: 0.3197 - val_loss: 0.2430 - val_accuracy: 0.2886
Epoch 7/10
3042/3042 [==============================] - 552s 182ms/step - loss: 0.0589 - accuracy: 0.3300 - val_

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2902, 1966, 29, 16, 2902, 10549, 463, 691, 2456, 1, 1, 193, 13, 16, 463, 2902, 1029, 463, 10549, 143, 463, 2902, 125, 16, 463, 2902, 1029, 463, 10549, 18, 40380, 44565, 17258, 22792, 30394, 324, 14, 193, 585, 463, 2902, 1642, 463, 2902, 463, 11711, 143, 463, 660, 463, 660, 1, 193, 16, 496,

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[1.27407666e-05, 1.92460247e-05, 3.75421700e-06, 1.56254464e-05,
        7.49512808e-03, 8.91838572e-04, 2.43572682e-08, 1.17978821e-11,
        4.99668090e-07],
       [1.05786944e-06, 1.82082913e-05, 7.27212495e-08, 3.94123902e-07,
        9.47720662e-04, 4.52208005e-05, 9.76817405e-10, 1.59236756e-14,
        2.87737052e-08],
       [6.40791235e-03, 3.76374461e-04, 4.97986786e-02, 1.15004637e-04,
        3.06370556e-02, 2.84878165e-02, 1.24838516e-06, 5.24166275e-08,
        3.41443993e-05],
       [1.12960013e-04, 3.63768893e-03, 1.15626735e-05, 3.77027638e-04,
        9.95574892e-02, 3.80139947e-02, 1.42799865e-06, 7.60835217e-09,
        1.85856934e-05],
       [1.08500026e-04, 2.81553133e-04, 1.31015383e-07, 2.56664280e-05,
        1.76877074e-03, 1.28349755e-02, 7.99208408e-07, 2.78592677e-10,
        1.04441483e-06],
       [3.31044674e-01, 3.77892256e-01, 1.76035818e-02, 2.76904623e-03,
        9.87095714e-01, 1.43798083e-01, 3.61644311e-06, 1.84746327e-07,
        1.7

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[1.2740767e-05 1.9246025e-05 3.7542170e-06 ... 2.4357268e-08
  1.1797882e-11 4.9966809e-07]
 [1.0578694e-06 1.8208291e-05 7.2721249e-08 ... 9.7681740e-10
  1.5923676e-14 2.8773705e-08]
 [6.4079124e-03 3.7637446e-04 4.9798679e-02 ... 1.2483852e-06
  5.2416627e-08 3.4144399e-05]
 ...
 [5.8034700e-03 3.0689418e-01 2.7814554e-03 ... 1.9734941e-04
  3.4944951e-05 7.7914627e-04]
 [2.7362925e-05 9.9023730e-01 9.9999797e-01 ... 1.3933160e-14
  3.9197776e-16 1.0307557e-06]
 [2.9457992e-06 1.2869172e-05 3.1522066e-07 ... 5.3575281e-08
  1.1390768e-12 1.2548259e-06]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.0431 with F-Score: 0.6839
permission  1
Length of sequence: 10000
Best Threshold: 0.0855 with F-Score: 0.5655
permission  2
Length of sequence: 10000
Best Threshold: 0.0499 with F-Score: 0.5832
permission  3
Length of sequence: 10000
Best Threshold: 0.0237 with F-Score: 0.5344
permission  4
Length of sequence: 10000
Best Threshold: 0.0202 with F-Score: 0.7875
permission  5
Length of sequence: 10000
Best Threshold: 0.1014 with F-Score: 0.5264
permission  6
Length of sequence: 10000
Best Threshold: 0.0069 with F-Score: 0.0476
permission  7
Length of sequence: 10000
Best Threshold: 0.3227 with F-Score: 0.4444
permission  8
Length of sequence: 10000
Best Threshold: 0.0842 with F-Score: 0.4179
-------------------------------------
optimal threshold tuning for f-score
[0.0431, 0.0855, 0.0499, 0.0237, 0.0202, 0.1014, 0.0069, 0.3227, 0.0842]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.006899999920278788 with G-Mean: 0.845
FPR: 0.125, TPR: 0.8161
permission  1
Best Threshold: 0.004399999976158142 with G-Mean: 0.7741
FPR: 0.2433, TPR: 0.7919
permission  2
Best Threshold: 0.0010000000474974513 with G-Mean: 0.8215
FPR: 0.1664, TPR: 0.8096
permission  3
Best Threshold: 0.0010000000474974513 with G-Mean: 0.7915
FPR: 0.2071, TPR: 0.79
permission  4
Best Threshold: 0.06430000066757202 with G-Mean: 0.775
FPR: 0.2322, TPR: 0.7823
permission  5
Best Threshold: 0.04520000144839287 with G-Mean: 0.7655
FPR: 0.1782, TPR: 0.7132
permission  6
Best Threshold: 0.0 with G-Mean: 0.8301
FPR: 0.1578, TPR: 0.8182
permission  7
Best Threshold: 0.0 with G-Mean: 0.9419
FPR: 0.1128, TPR: 1.0
permission  8
Best Threshold: 9.999999747378752e-05 with G-Mean: 0.8253
FPR: 0.2338, TPR: 0.8889
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0069, 0.0044, 0.001, 0.001, 0.0643, 0.0452, 0.0, 0.0, 1e-04]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6839
  Location  : 0.5655
  Microphone: 0.5832
  Contacts  : 0.5344
  Storage   : 0.7875
  Phone     : 0.5264
  SMS       : 0.0476
  Call_Log  : 0.4444
  Calendar  : 0.4179

  Average F1 score: 0.5101


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8968
  Location  : 0.8594
  Microphone: 0.9215
  Contacts  : 0.8904
  Storage   : 0.7582
  Phone     : 0.8428
  SMS       : 0.9827
  Call_Log  : 0.9989
  Calendar  : 0.9831

  Average F1 (micro) score: 0.9038


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8442
  Location  : 0.7739
  Microphone: 0.8206
  Contacts  : 0.7887
  Storage   : 0.7748
  Phone     : 0.7667
  SMS       : 0.5000
  Call_Log  : 0.5000
  Calendar  : 0.8221

  Average ROC_AUC score: 0.7323


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.0313 with F-Score: 0.4594
permission  1
Length of sequence: 10000
Best Threshold: 0.0443 with F-Score: 0.5428
permission  2
Length of sequence: 10000
Best Threshold: 0.1377 with F-Score: 0.5705
permission  3
Length of sequence: 10000
Best Threshold: 0.0016 with F-Score: 0.5663
permission  4
Length of sequence: 10000
Best Threshold: 0.0005 with F-Score: 0.581
permission  5
Length of sequence: 10000
Best Threshold: 0.729 with F-Score: 0.4426
permission  6
Length of sequence: 10000
Best Threshold: 0.0005 with F-Score: 0.4912
permission  7
Length of sequence: 10000
Best Threshold: 0.0062 with F-Score: 0.3568
permission  8
Length of sequence: 10000
Best Threshold: 0.0081 with F-Score: 0.4588
-------------------------------------
optimal threshold tuning for f-score
[0.0313, 0.0443, 0.1377, 0.0016, 0.0005, 0.729, 0.0005, 0.0062, 0.0081]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.003000000026077032 with G-Mean: 0.7021
FPR: 0.3586, TPR: 0.7686
permission  1
Best Threshold: 0.03319999948143959 with G-Mean: 0.7263
FPR: 0.2577, TPR: 0.7105
permission  2
Best Threshold: 0.007499999832361937 with G-Mean: 0.7922
FPR: 0.1556, TPR: 0.7432
permission  3
Best Threshold: 0.0015999999595806003 with G-Mean: 0.681
FPR: 0.3207, TPR: 0.6828
permission  4
Best Threshold: 0.08420000225305557 with G-Mean: 0.569
FPR: 0.4456, TPR: 0.5841
permission  5
Best Threshold: 0.09399999678134918 with G-Mean: 0.7307
FPR: 0.3162, TPR: 0.7807
permission  6
Best Threshold: 0.0 with G-Mean: 0.7492
FPR: 0.3278, TPR: 0.8349
permission  7
Best Threshold: 0.0 with G-Mean: 0.7237
FPR: 0.2382, TPR: 0.6875
permission  8
Best Threshold: 0.00139999995008111 with G-Mean: 0.8201
FPR: 0.1771, TPR: 0.8173
-------------------------------------
ROC curve with G-mean threshold tuning
[0.003, 0.0332, 0.0075, 0.0016, 0.0842, 0.094, 0.0, 0.0, 0.0014]


#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4594
  Location  : 0.5428
  Microphone: 0.5705
  Contacts  : 0.5663
  Storage   : 0.5810
  Phone     : 0.4426
  SMS       : 0.4912
  Call_Log  : 0.3568
  Calendar  : 0.4588

  Average F1 score: 0.4966


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.7836
  Location  : 0.7546
  Microphone: 0.9074
  Contacts  : 0.6782
  Storage   : 0.4187
  Phone     : 0.9074
  SMS       : 0.8373
  Call_Log  : 0.9158
  Calendar  : 0.8932

  Average F1 (micro) score: 0.7885


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV4_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7032
  Location  : 0.7264
  Microphone: 0.7904
  Contacts  : 0.6795
  Storage   : 0.5683
  Phone     : 0.7279
  SMS       : 0.5000
  Call_Log  : 0.5000
  Calendar  : 0.8193

  Average ROC_AUC score: 0.6683
